In [7]:
from os import walk

dirpath = "ABox";
filenames = next(walk(dirpath), (None, None, []))[2]  # [] if no file
print(filenames)

['C_B307_Brick.ttl', 'C_B307_DigitalBuilding.ttl', 'C_B307_Haystack.ttl', 'C_B307_RECore.ttl']


In [8]:
from rdflib import Graph as RDFGraph
from rdflib.extras.external_graph_libs import rdflib_to_networkx_digraph

# loadn RDF Graph
dic = {}
for filename in filenames:
    print(filename)
    path = dirpath + "/" + filename
    g = RDFGraph()
    g.parse(path)
    print("rdflib Graph loaded successfully with {} triples".format(len(g)))
    G = rdflib_to_networkx_digraph(g)
    print("networkx Graph loaded successfully with length {}".format(len(G)))
    dic.update({str(filename):G})
# print (dic)

C_B307_Brick.ttl
rdflib Graph loaded successfully with 1618 triples
networkx Graph loaded successfully with length 831
C_B307_DigitalBuilding.ttl
rdflib Graph loaded successfully with 929 triples
networkx Graph loaded successfully with length 542
C_B307_Haystack.ttl
rdflib Graph loaded successfully with 1266 triples
networkx Graph loaded successfully with length 652
C_B307_RECore.ttl
rdflib Graph loaded successfully with 1843 triples
networkx Graph loaded successfully with length 776


In [9]:
# show complete node
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [ ]:
nx.simrank_similarity(G, source=0, target=0)

In [4]:
number_of_nodes = {}
for k, v in dic.items():
    v = v.number_of_nodes()
    number_of_nodes.update({k:v})

number_of_nodes_df = pd.DataFrame(data = number_of_nodes,index=[0])
number_of_nodes_df.head(10)

,C_B307_Brick.ttl,C_B307_DigitalBuilding.ttl,C_B307_Haystack.ttl,C_B307_RECore.ttl
0,831,542,652,776


In [17]:
import networkx as nx
import math
from rdflib import URIRef
for k, v in dic.items():
    a= nx.panther_similarity(v, URIRef('http://design-pattern-instance-only.qzc438.com/Brick_1.2.1/C_B307_Brick#AHU_F1'))
    print(a)

ValueError: probabilities contain NaN

In [6]:
--NotebookApp.iopub_data_rate_limit=1.0e10

SyntaxError: can't assign to operator (3512861074.py, line 1)

In [ ]:
number_of_edges = {}
for k, v in dic.items():
    v = v.number_of_edges()
    number_of_edges.update ({k:v})

number_of_edges_df = pd.DataFrame(data = number_of_edges,index=[0])
number_of_edges_df.head(10)

In [ ]:
prefix_brick = "http://design-pattern-instance-only.qzc438.com/Brick_1.2.1/C_B307_Brick#"
prefix_haystack = "http://design-pattern-instance-only.qzc438.com/ProjectHaystack/C_B307_Haystack#"
prefix_recore = "http://design-pattern-instance-only.qzc438.com/RealEstateCore/C_B307_RECore#"
prefix_digitalbuilding = "http://design-pattern-instance-only.qzc438.com/GoogleDigitalBuilding/C_B307_DigitalBuilding#"
prefix_test = "dch:org/csiro/site/Clayton/building/307_R_L_Sandland#"
prefix = [prefix_brick, prefix_haystack, prefix_recore, prefix_digitalbuilding]

### Dan's Paper (degree and class hierarchy/depth)

In [ ]:
import networkx as nx

dan_summary_brick_df = pd.DataFrame()
dan_summary_haystack_df = pd.DataFrame()
dan_summary_recore_df = pd.DataFrame()
dan_summary_digitalbuilding_df = pd.DataFrame()

In [ ]:
# number of degree
for k, v in dic.items():
    number_of_degree_df = pd.DataFrame(data = v.degree(), 
                                           columns = ["node", "number_of_degree"])
    # filter
    if k == "C_B307_Brick.ttl":
        number_of_degree_df = number_of_degree_df[number_of_degree_df['node'].str.startswith(prefix_brick)]
        dan_summary_brick_df = number_of_degree_df
    if k == "C_B307_Haystack.ttl":
        number_of_degree_df = number_of_degree_df[number_of_degree_df['node'].str.startswith(prefix_haystack)]
        dan_summary_haystack_df = number_of_degree_df
    if k == "C_B307_RECore.ttl":
        number_of_degree_df = number_of_degree_df[number_of_degree_df['node'].str.startswith(prefix_recore)]
        dan_summary_recore_df = number_of_degree_df
    if k == "C_B307_DigitalBuilding.ttl":
        number_of_degree_df = number_of_degree_df[number_of_degree_df['node'].str.startswith(prefix_digitalbuilding)]
        dan_summary_digitalbuilding_df = number_of_degree_df
#     if k == "C_B307_test.ttl":
#         numbr_of_degree_df = numbr_of_degree_df[numbr_of_degree_df['node'].str.startswith(prefix_test)]
    
    # sort
    number_of_degree_df = number_of_degree_df.sort_values("number_of_degree", ascending = False)
    # reset index
    number_of_degree_df = number_of_degree_df.reset_index(drop=True)
    
    # print
    print ("file:", k)
    print("data points", len(number_of_degree_df.index))
    print (number_of_degree_df.head(5))
    print ("====================================================================================================")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
x1 = dan_summary_brick_df.number_of_degree
x2 = dan_summary_haystack_df.number_of_degree
x3 = dan_summary_recore_df.number_of_degree
x4 = dan_summary_digitalbuilding_df.number_of_degree
plt.boxplot ([x1,x2,x3,x4])
plt.xticks([1,2,3,4], ['Brick','Haystack','RECore','DigitalBuilding'])
plt.title ("Number of Degrees (Original Model)")
plt.savefig('Number of Degrees-Original.jpg', bbox_inches='tight', dpi=500)

In [ ]:
# start from root to each node, calculate the depth for each node
from rdflib import URIRef
# class hierarchy/depth
for k, v in dic.items():
    
    # filter
    if k == "C_B307_Brick.ttl":        
        
        root = URIRef('http://design-pattern-instance-only.qzc438.com/Brick_1.2.1/C_B307_Brick#AHU_F1')
        
        data = {}
        
        for i,j in v.nodes(data=True):
                
            paths = nx.all_simple_paths(v, source = root, target = i)
            
            if(len(list(paths)) == 0):
                depth = 0
                # data.update({i:depth})
            else:
                depth = len(max(nx.all_simple_paths(v, source = root, target = i), key=lambda x: len(x)))-1
                data.update({i:depth})
            
        class_hierarchy_df = pd.DataFrame.from_dict(data = data, orient='index', columns = ["class_hierarchy"])
        class_hierarchy_df.index.rename('node', inplace=True)
        class_hierarchy_df = class_hierarchy_df.reset_index()
        # filter
        class_hierarchy_df = class_hierarchy_df[class_hierarchy_df['node'].str.startswith(prefix_brick)]
        # sort
        class_hierarchy_df = class_hierarchy_df.sort_values("class_hierarchy", ascending = False)
        # reset index
        class_hierarchy_df = class_hierarchy_df.reset_index(drop=True)
        # merge
        dan_summary_brick_df = class_hierarchy_df

    
    if k == "C_B307_Haystack.ttl":        

        root = URIRef('http://design-pattern-instance-only.qzc438.com/ProjectHaystack/C_B307_Haystack#AHU_F1')

        data = {}

        for i,j in v.nodes(data=True):

            paths = nx.all_simple_paths(v, source = root, target = i)

            if(len(list(paths)) == 0):
                depth = 0
                # data.update({i:depth})
            else:
                depth = len(max(nx.all_simple_paths(v, source = root, target = i), key=lambda x: len(x)))-1
                data.update({i:depth})

        class_hierarchy_df = pd.DataFrame.from_dict(data = data, orient='index', columns = ["class_hierarchy"])
        class_hierarchy_df.index.rename('node', inplace=True)
        class_hierarchy_df = class_hierarchy_df.reset_index()
        # filter
        class_hierarchy_df = class_hierarchy_df[class_hierarchy_df['node'].str.startswith(prefix_haystack)]
        # sort
        class_hierarchy_df = class_hierarchy_df.sort_values("class_hierarchy", ascending = False)
        # reset index
        class_hierarchy_df = class_hierarchy_df.reset_index(drop=True)
        # merge
        dan_summary_haystack_df = class_hierarchy_df
    
    if k == "C_B307_RECore.ttl":        

        root = URIRef('http://design-pattern-instance-only.qzc438.com/RealEstateCore/C_B307_RECore#AHU_F1')

        data = {}

        for i,j in v.nodes(data=True):

            paths = nx.all_simple_paths(v, source = root, target = i)
            
            if(len(list(paths)) == 0):
                depth = 0
                # data.update({i:depth})
            else:
                depth = len(max(nx.all_simple_paths(v, source = root, target = i), key=lambda x: len(x)))-1
                data.update({i:depth})

        class_hierarchy_df = pd.DataFrame.from_dict(data = data, orient='index', columns = ["class_hierarchy"])
        class_hierarchy_df.index.rename('node', inplace=True)
        class_hierarchy_df = class_hierarchy_df.reset_index()
        # filter
        class_hierarchy_df = class_hierarchy_df[class_hierarchy_df['node'].str.startswith(prefix_recore)]
        # sort
        class_hierarchy_df = class_hierarchy_df.sort_values("class_hierarchy", ascending = False)
        # reset index
        class_hierarchy_df = class_hierarchy_df.reset_index(drop=True)
        # merge
        dan_summary_recore_df = class_hierarchy_df
        
    if k == "C_B307_DigitalBuilding.ttl":        

        root = URIRef('http://design-pattern-instance-only.qzc438.com/GoogleDigitalBuilding/C_B307_DigitalBuilding#AHU_F1')

        data = {}

        for i,j in v.nodes(data=True):

            paths = nx.all_simple_paths(v, source = root, target = i)
            
            if(len(list(paths)) == 0):
                depth = 0
                # data.update({i:depth})
            else:
                depth = len(max(nx.all_simple_paths(v, source = root, target = i), key=lambda x: len(x)))-1
                data.update({i:depth})

        class_hierarchy_df = pd.DataFrame.from_dict(data = data, orient='index', columns = ["class_hierarchy"])
        class_hierarchy_df.index.rename('node', inplace=True)
        class_hierarchy_df = class_hierarchy_df.reset_index()
        # filter
        class_hierarchy_df = class_hierarchy_df[class_hierarchy_df['node'].str.startswith(prefix_digitalbuilding)]
        # sort
        class_hierarchy_df = class_hierarchy_df.sort_values("class_hierarchy", ascending = False)
        # reset index
        class_hierarchy_df = class_hierarchy_df.reset_index(drop=True)
        # merge
        dan_summary_digitalbuilding_df = class_hierarchy_df
    
    # print
    print ("file:", k)
    print("data points", len(class_hierarchy_df.index))
    print (class_hierarchy_df.head(10))
    print ("====================================================================================================")

In [ ]:
dan_summary_brick_df

In [ ]:
x1 = dan_summary_brick_df.class_hierarchy
x2 = dan_summary_haystack_df.class_hierarchy
x3 = dan_summary_recore_df.class_hierarchy
x4 = dan_summary_digitalbuilding_df.class_hierarchy
plt.boxplot ([x1,x2,x3,x4])
plt.xticks([1,2,3,4], ['Brick','Haystack','RECore','DigitalBuilding'])
plt.title ("Class Hierarchy (Original Model)")
plt.savefig('Class Hierarchy-Original.jpg', bbox_inches='tight', dpi=500)

In [ ]:
# testbed
from rdflib import URIRef
# class hierarchy
for k, v in dic.items():
    
    # filter
    if k == "C_B307_Brick.ttl":
        
        a = URIRef('http://design-pattern-instance-only.qzc438.com/Brick_1.2.1/C_B307_Brick#Building307')
        
        print(v.has_node(a))
        
        
        nx.shortest_path_length(v,a)
        
        path = dict(nx.all_pairs_shortest_path(v))
        
        aa = {}
        for key, value in path[URIRef('http://design-pattern-instance-only.qzc438.com/Brick_1.2.1/C_B307_Brick#Building307')].items():
            aa.update({key:len([item for item in value if item])-1})
        print(aa)
            
        numbr_of_degree_df = pd.DataFrame.from_dict(data = aa, orient='index', columns = ["numbr_of_degree"])
    
        numbr_of_degree_df.index.rename('node', inplace=True)
        numbr_of_degree_df = numbr_of_degree_df.reset_index()
    
        print (numbr_of_degree_df)

In [ ]:
# alternative way to calculate
from rdflib import URIRef
# class hierarchy
for k, v in dic.items():
    
    # filter
    if k == "C_B307_Brick.ttl":
        
        path = dict(nx.all_pairs_shortest_path(v))
        
        aa = {}
        for key, value in path[URIRef('http://design-pattern-instance-only.qzc438.com/Brick_1.2.1/C_B307_Brick#Building307')].items():
            aa.update({key:len([item for item in value if item])-1})
        print(aa)
            
        numbr_of_degree_df = pd.DataFrame.from_dict(data = aa, orient='index', columns = ["numbr_of_degree"])
    
        numbr_of_degree_df.index.rename('node', inplace=True)
        numbr_of_degree_df = numbr_of_degree_df.reset_index()
    
        print (numbr_of_degree_df)

### Centrality Analysis

### 1. Heuristic (simple) centrality: Degree centrality (local importance)

In [ ]:
import networkx as nx

centrality_summary_brick_df = pd.DataFrame()
centrality_summary_haystack_df = pd.DataFrame()
centrality_summary_recore_df = pd.DataFrame()
centrality_summary_digitalbuilding_df = pd.DataFrame()

In [ ]:
# in-degree centrality
for k, v in dic.items():
    in_degree_centrality_df = pd.DataFrame(data = nx.in_degree_centrality(v).items(), 
                                           columns = ["node", "in_degree_centrality"])
#     print (in_degree_centrality_df.head(10))
    # filter
    if k == "C_B307_Brick.ttl":
        in_degree_centrality_df = in_degree_centrality_df[in_degree_centrality_df['node'].str.startswith(prefix_brick)]
        centrality_summary_brick_df = in_degree_centrality_df
    if k == "C_B307_Haystack.ttl":
        in_degree_centrality_df = in_degree_centrality_df[in_degree_centrality_df['node'].str.startswith(prefix_haystack)]
        centrality_summary_haystack_df = in_degree_centrality_df
    if k == "C_B307_RECore.ttl":
        in_degree_centrality_df = in_degree_centrality_df[in_degree_centrality_df['node'].str.startswith(prefix_recore)]
        centrality_summary_recore_df = in_degree_centrality_df
    if k == "C_B307_DigitalBuilding.ttl":
        in_degree_centrality_df = in_degree_centrality_df[in_degree_centrality_df['node'].str.startswith(prefix_digitalbuilding)]
        centrality_summary_digitalbuilding_df = in_degree_centrality_df
    
    # sort
    in_degree_centrality_df = in_degree_centrality_df.sort_values("in_degree_centrality", ascending = False)
    
    # print
    print (k)
    print (in_degree_centrality_df.head(10))
    print ("==================================================")

In [ ]:
# out-degree centrality
for k, v in dic.items():
    out_degree_centrality_df = pd.DataFrame(data = nx.out_degree_centrality(v).items(), 
                                           columns = ["node", "out_degree_centrality"])
    # filter
    if k == "C_B307_Brick.ttl":
        out_degree_centrality_df = out_degree_centrality_df[out_degree_centrality_df['node'].str.startswith(prefix_brick)]
        centrality_summary_brick_df = centrality_summary_brick_df.merge(out_degree_centrality_df, on="node")
    if k == "C_B307_Haystack.ttl":
        out_degree_centrality_df = out_degree_centrality_df[out_degree_centrality_df['node'].str.startswith(prefix_haystack)]
        centrality_summary_haystack_df = centrality_summary_haystack_df.merge(out_degree_centrality_df, on="node")
    if k == "C_B307_RECore.ttl":
        out_degree_centrality_df = out_degree_centrality_df[out_degree_centrality_df['node'].str.startswith(prefix_recore)]
        centrality_summary_recore_df = centrality_summary_recore_df.merge(out_degree_centrality_df, on="node")
    if k == "C_B307_DigitalBuilding.ttl":
        out_degree_centrality_df = out_degree_centrality_df[out_degree_centrality_df['node'].str.startswith(prefix_digitalbuilding)]
        centrality_summary_digitalbuilding_df = centrality_summary_digitalbuilding_df.merge(out_degree_centrality_df, on="node")
    
    # sort
    out_degree_centrality_df = out_degree_centrality_df.sort_values("out_degree_centrality", ascending = False)
    
    # print
    print (k)
    print (out_degree_centrality_df.head(10))
    print ("==================================================")

## 2. Graphical centrality

### (1) Closeness centrality (path spread importance)

In [ ]:
# closeness centrality
for k, v in dic.items():
    closeness_centrality_df = pd.DataFrame(data = nx.closeness_centrality(v).items(), 
                                           columns = ["node", "closeness_centrality"])
    # filter
    if k == "C_B307_Brick.ttl":
        closeness_centrality_df = closeness_centrality_df[closeness_centrality_df['node'].str.startswith(prefix_brick)]
        centrality_summary_brick_df = centrality_summary_brick_df.merge(closeness_centrality_df, on="node")
    if k == "C_B307_Haystack.ttl":
        closeness_centrality_df = closeness_centrality_df[closeness_centrality_df['node'].str.startswith(prefix_haystack)]
        centrality_summary_haystack_df = centrality_summary_haystack_df.merge(closeness_centrality_df, on="node")
    if k == "C_B307_RECore.ttl":
        closeness_centrality_df = closeness_centrality_df[closeness_centrality_df['node'].str.startswith(prefix_recore)]
        centrality_summary_recore_df = centrality_summary_recore_df.merge(closeness_centrality_df, on="node")
    if k == "C_B307_DigitalBuilding.ttl":
        closeness_centrality_df = closeness_centrality_df[closeness_centrality_df['node'].str.startswith(prefix_digitalbuilding)]
        centrality_summary_digitalbuilding_df = centrality_summary_digitalbuilding_df.merge(closeness_centrality_df, on="node")
    
    # sort
    closeness_centrality_df = closeness_centrality_df.sort_values("closeness_centrality", ascending = False)
    
    # print
    print (k)
    print (closeness_centrality_df.head(10))
    print ("==================================================")

### (2) Betweenness centrality (In-between/broker/bridge/middleman importance)

In [ ]:
# betweenness centrality
for k, v in dic.items():
    betweenness_centrality_df = pd.DataFrame(data = nx.betweenness_centrality(v).items(), 
                                           columns = ["node", "betweenness_centrality"])
    # filter
    if k == "C_B307_Brick.ttl":
        betweenness_centrality_df = betweenness_centrality_df[betweenness_centrality_df['node'].str.startswith(prefix_brick)]
        centrality_summary_brick_df = centrality_summary_brick_df.merge(betweenness_centrality_df, on="node")
    if k == "C_B307_Haystack.ttl":
        betweenness_centrality_df = betweenness_centrality_df[betweenness_centrality_df['node'].str.startswith(prefix_haystack)]
        centrality_summary_haystack_df = centrality_summary_haystack_df.merge(betweenness_centrality_df, on="node")
    if k == "C_B307_RECore.ttl":
        betweenness_centrality_df = betweenness_centrality_df[betweenness_centrality_df['node'].str.startswith(prefix_recore)]
        centrality_summary_recore_df = centrality_summary_recore_df.merge(betweenness_centrality_df, on="node")
    if k == "C_B307_DigitalBuilding.ttl":
        betweenness_centrality_df = betweenness_centrality_df[betweenness_centrality_df['node'].str.startswith(prefix_digitalbuilding)]
        centrality_summary_digitalbuilding_df = centrality_summary_digitalbuilding_df.merge(betweenness_centrality_df, on="node")
    
    # sort
    betweenness_centrality_df = betweenness_centrality_df.sort_values("betweenness_centrality", ascending = False)
    
    # print
    print (k)
    print (betweenness_centrality_df.head(10))
    print ("==================================================")

## 3. Logical centrality

###  (1) PageRank centrality

In [ ]:
# pagerank centrality
for k, v in dic.items():
    pagerank_centrality_df = pd.DataFrame(data = nx.pagerank(v).items(), 
                                           columns = ["node", "pagerank_centrality"])
    # filter
    if k == "C_B307_Brick.ttl":
        pagerank_centrality_df = pagerank_centrality_df[pagerank_centrality_df['node'].str.startswith(prefix_brick)]
        centrality_summary_brick_df = centrality_summary_brick_df.merge(pagerank_centrality_df, on="node")
    if k == "C_B307_Haystack.ttl":
        pagerank_centrality_df = pagerank_centrality_df[pagerank_centrality_df['node'].str.startswith(prefix_haystack)]
        centrality_summary_haystack_df = centrality_summary_haystack_df.merge(pagerank_centrality_df, on="node")
    if k == "C_B307_RECore.ttl":
        pagerank_centrality_df = pagerank_centrality_df[pagerank_centrality_df['node'].str.startswith(prefix_recore)]
        centrality_summary_recore_df = centrality_summary_recore_df.merge(pagerank_centrality_df, on="node")
    if k == "C_B307_DigitalBuilding.ttl":
        pagerank_centrality_df = pagerank_centrality_df[pagerank_centrality_df['node'].str.startswith(prefix_digitalbuilding)]
        centrality_summary_digitalbuilding_df = centrality_summary_digitalbuilding_df.merge(pagerank_centrality_df, on="node")
    
    # sort
    pagerank_centrality_df = pagerank_centrality_df.sort_values("pagerank_centrality", ascending = False)
    
    # print
    print (k)
    print (pagerank_centrality_df.head(10))
    print ("==================================================")
    
    print(centrality_summary_digitalbuilding_df)

### (2) Hit: Authority and Hub scores (inward and outward importance)

In [ ]:
for k, v in dic.items():
    # hits
    hub_scores, auth_scores = nx.hits(v)
    
    # hub score
    hub_centrality_df = pd.DataFrame(data = hub_scores.items(), 
                                           columns = ["node", "hub_centrality"])
    # filter
    if k == "C_B307_Brick.ttl":
        hub_centrality_df = hub_centrality_df[hub_centrality_df['node'].str.startswith(prefix_brick)]
        centrality_summary_brick_df = centrality_summary_brick_df.merge(hub_centrality_df, on="node")
    if k == "C_B307_Haystack.ttl":
        hub_centrality_df = hub_centrality_df[hub_centrality_df['node'].str.startswith(prefix_haystack)]
        centrality_summary_haystack_df = centrality_summary_haystack_df.merge(hub_centrality_df, on="node")
    if k == "C_B307_RECore.ttl":
        hub_centrality_df = hub_centrality_df[hub_centrality_df['node'].str.startswith(prefix_recore)]
        centrality_summary_recore_df = centrality_summary_recore_df.merge(hub_centrality_df, on="node")
    if k == "C_B307_DigitalBuilding.ttl":
        hub_centrality_df = hub_centrality_df[hub_centrality_df['node'].str.startswith(prefix_digitalbuilding)]
        centrality_summary_digitalbuilding_df = centrality_summary_digitalbuilding_df.merge(hub_centrality_df, on="node")
    
    # sort
    hub_centrality_df = hub_centrality_df.sort_values("hub_centrality", ascending = False)
    
    # print
    print (k)
    print (hub_centrality_df.head(10))
    print ("==================================================")
    
    
    # authority score
    auth_centrality_df = pd.DataFrame(data = auth_scores.items(), 
                                           columns = ["node", "auth_centrality"])
    # filter
    if k == "C_B307_Brick.ttl":
        auth_centrality_df = auth_centrality_df[auth_centrality_df['node'].str.startswith(prefix_brick)]
        centrality_summary_brick_df = centrality_summary_brick_df.merge(auth_centrality_df, on="node")
    if k == "C_B307_Haystack.ttl":
        auth_centrality_df = auth_centrality_df[auth_centrality_df['node'].str.startswith(prefix_haystack)]
        centrality_summary_haystack_df = centrality_summary_haystack_df.merge(auth_centrality_df, on="node")
    if k == "C_B307_RECore.ttl":
        auth_centrality_df = auth_centrality_df[auth_centrality_df['node'].str.startswith(prefix_recore)]
        centrality_summary_recore_df = centrality_summary_recore_df.merge(auth_centrality_df, on="node")
    if k == "C_B307_DigitalBuilding.ttl":
        auth_centrality_df = auth_centrality_df[auth_centrality_df['node'].str.startswith(prefix_digitalbuilding)]
        centrality_summary_digitalbuilding_df = centrality_summary_digitalbuilding_df.merge(auth_centrality_df, on="node")
    
    # sort
    auth_centrality_df = auth_centrality_df.sort_values("auth_centrality", ascending = False)
    
    # print
    print (k)
    print (auth_centrality_df.head(10))
    print ("==================================================")
    
    print(centrality_summary_digitalbuilding_df)

In [ ]:
print(centrality_summary_brick_df)

In [ ]:
centrality_ranks = centrality_summary_brick_df.rank(ascending=False, 
                                           numeric_only=True, 
                                           method="dense").astype(int)
centrality_ranks.insert(loc=0, column='node', value=centrality_summary_brick_df["node"])
centrality_ranks.sort_values("in_degree_centrality").style.background_gradient(subset=list(centrality_ranks.columns[1:]))

In [ ]:
centrality_ranks = centrality_summary_haystack_df.rank(ascending=False, 
                                           numeric_only=True, 
                                           method="dense").astype(int)
centrality_ranks.insert(loc=0, column='node', value=centrality_summary_brick_df["node"])
centrality_ranks.sort_values("in_degree_centrality").style.background_gradient(subset=list(centrality_ranks.columns[1:]))

In [ ]:
from openpyxl import Workbook

with pd.ExcelWriter('pandas_to_excel.xlsx') as writer:
    centrality_ranks.to_excel(writer, sheet_name='sheet1')

In [ ]:
# compute the average ranking using the above ranks
average_ranks = pd.DataFrame(round(centrality_ranks.mean(axis=1)).astype(int), columns=["average_rank"])
average_ranks.insert(loc=0, column='node', value=centrality_summary_brick_df["node"])
average_ranks.sort_values("average_rank").reset_index(drop=True)\
    .style.background_gradient(subset=["average_rank"])

In [ ]:
df = pd.DataFrame(columns = ['node','in_degree_centrality','out_degree_centrality','closeness_centrality','betweenness_centrality','pagerank_centrality','hub_centrality','hub_centrality'])

centrality_summary_brick_sensor = centrality_summary_brick_df.loc[centrality_summary_brick_df.node.str.strip() 
                                == "http://design-pattern-instance-only.qzc438.com/Brick_1.2.1/C_B307_Brick#Chiller1"]
centrality_summary_brick_sensor.style.background_gradient(cmap='Blues')
centrality_summary_haystack_sensor = centrality_summary_haystack_df.loc[centrality_summary_haystack_df.node.str.strip() 
                                == "http://design-pattern-instance-only.qzc438.com/ProjectHaystack/C_B307_Haystack#Chiller1"]
centrality_summary_recore_sensor = centrality_summary_recore_df.loc[centrality_summary_recore_df.node.str.strip() 
                                == "http://design-pattern-instance-only.qzc438.com/RealEstateCore/C_B307_RECore#Chiller1"]
centrality_summary_digitalbuilding_sensor = centrality_summary_digitalbuilding_df.loc[centrality_summary_digitalbuilding_df.node.str.strip() 
                                == "http://design-pattern-instance-only.qzc438.com/GoogleDigitalBuilding/C_B307_DigitalBuilding#Chiller1"]

df = pd.concat([centrality_summary_brick_sensor,centrality_summary_haystack_sensor,centrality_summary_recore_sensor,centrality_summary_digitalbuilding_sensor])
# df.style.background_gradient(subset=list(df.columns[1:]))
df

In [ ]:
df = pd.DataFrame(columns = ['node','in_degree_centrality','out_degree_centrality','closeness_centrality','betweenness_centrality','pagerank_centrality','hub_centrality','hub_centrality'])

centrality_summary_brick_sensor = centrality_summary_brick_df.loc[centrality_summary_brick_df.node.str.strip() 
                                == "http://design-pattern-instance-only.qzc438.com/Brick_1.2.1/C_B307_Brick#G6"]
centrality_summary_brick_sensor.style.background_gradient(cmap='Blues')
centrality_summary_haystack_sensor = centrality_summary_haystack_df.loc[centrality_summary_haystack_df.node.str.strip() 
                                == "http://design-pattern-instance-only.qzc438.com/ProjectHaystack/C_B307_Haystack#G6"]
centrality_summary_recore_sensor = centrality_summary_recore_df.loc[centrality_summary_recore_df.node.str.strip() 
                                == "http://design-pattern-instance-only.qzc438.com/RealEstateCore/C_B307_RECore#G6"]
centrality_summary_digitalbuilding_sensor = centrality_summary_digitalbuilding_df.loc[centrality_summary_digitalbuilding_df.node.str.strip() 
                                == "http://design-pattern-instance-only.qzc438.com/GoogleDigitalBuilding/C_B307_DigitalBuilding#G6"]

df = pd.concat([centrality_summary_brick_sensor,centrality_summary_haystack_sensor,centrality_summary_recore_sensor,centrality_summary_digitalbuilding_sensor])
# df.style.background_gradient(subset=list(df.columns[1:]))
df

In [ ]:
print(centrality_summary_recore_df)

In [ ]:
centrality_summary_recore_df.loc[centrality_summary_recore_df.node.str.strip() 
                                == "http://design-pattern-instance-only.qzc438.com/RealEstateCore/C_B307_RECore#Level1_1.111"]